In [1]:
from api_key import SERPER_API_KEY
import os
os.environ["SERPER_API_KEY"] = SERPER_API_KEY  # serper.dev API key

import litellm
litellm.set_verbose=True

# 设置ollama模型
# os.environ["OPENAI_API_BASE"] = 'http://192.168.3.155:11434' 
# os.environ["OPENAI_MODEL_NAME"] ='ollama_chat/qwen2.5:latest'
# os.environ["OPENAI_API_KEY"] ='EMPTY'

from crewai.llm import LLM
llm=LLM(
    model="ollama_chat/qwen2.5:latest", 
    base_url="http://localhost:11434")

from crewai import Agent
from crewai_tools import SerperDevTool
search_tool = SerperDevTool(n_results=5)

# 创建一个带有内存和冗长模式的高级研究员代理人
researcher = Agent(
    role='Senior Researcher',
    goal='在 {topic} 领域发现突破性技术',
    verbose=True,
    memory=True,
    backstory=(
        "出于好奇心驱使，您处于创新的最前沿，渴望探索和分享可能改变世界的知识。"
    ),
    llm=llm,
    tools=[search_tool],
    allow_delegation=True
)

# 创建一个作家代理人，具有自定义工具和委派能力
writer = Agent(
    role='Writer',
    goal='讲述关于 {topic} 的引人入胜的科技故事',
    verbose=True,
    memory=True,
    backstory=(
        "擅长简化复杂话题，您撰写引人入胜的叙述，吸引人并教育他人，在易于理解的方式中揭示新的发现。"
    ),
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

In [2]:
from crewai import Task

# 研究任务
research_task = Task(
  description=(
    "识别 {topic} 领域的下一个大趋势。"
    "专注于识别利弊以及整体叙述。"
    "您的最终报告应清楚地阐明关键点、市场机会和潜在风险。"
  ),
  expected_output='一份关于最新人工智能趋势的全面报告，共 3 段。',
  tools=[search_tool],
  agent=researcher,
)

# 具有语言模型配置的写作任务
write_task = Task(
  description=(
    "撰写一篇关于 {topic} 的有见地的文章。"
    "关注最新趋势及其对行业的影响。"
    "这篇文章应易于理解、引人入胜且积极向上。"
  ),
  expected_output='一篇关于 {topic} 进展的 4 段文章，格式为 markdown。',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # 输出定制的示例
)

In [3]:
from crewai import Crew, Process

# 以一些增强配置形成以技术为中心的团队
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential,  # 可选：顺序任务执行是默认设置
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True,
    verbose=True,
    manager_llm=[llm],
    embedder={
        "provider": "ollama",
        "config": {
            "model": 'quentinz/bge-large-zh-v1.5:latest',
            "base_url": 'http://localhost:11434'
        }
    }
)

In [4]:
# 以增强反馈开始任务执行流程
result = crew.kickoff(inputs={'topic': '医疗人工智能'})
print(result)

# Agent: Senior Researcher
## Task: 识别 医疗人工智能 领域的下一个大趋势。专注于识别利弊以及整体叙述。您的最终报告应清楚地阐明关键点、市场机会和潜在风险。


# Agent: Senior Researcher
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"\\u533b\\u7597\\u4eba\\u5de5\\u667a\\u80fd \\u6700\\u65b0\\u8d8b\\u52bf \\u5229\\u5f0a\\u5206\\u6790 \\u5e02\\u573a\\u524d\\u666f\"}"
## Tool Output: 

Search results: Title: 医疗保健市场规模中的人工智能[AI]分享，2032
Link: https://www.fortunebusinessinsights.com/zh/industry-reports/artificial-intelligence-in-healthcare-market-100534
Snippet: 预计该市场将从2024年的276.9亿美元增长到2032年的4909.6亿美元，期间复合年增长率为43.2%预测期。 医疗保健领域的人工智能(AI) 用于分析医疗数据，帮助 ...
---
Title: 医疗保健AI市场规模和份额分析-行业研究报告-增长趋势
Link: https://www.mordorintelligence.com/zh-CN/industry-reports/artificial-intelligence-in-medicine-market
Snippet: 2024年，医疗人工智能市场规模预计为116.6亿美元，预计到2029年将达到367.9亿美元，在预测期内（2024-2029年）复合年增长率为25.83%。 全球老年人口的不断增长、生活方式的变化 ...
---
Title: 人工智能在医疗领域的崛起与应用前景分析 - 续航教育
Link: https://www.forwardpathway.com/141671
Snippet: AI的潜力不仅限于提高诊断准确性，还包括优化治疗流程和改善患者的